In [1]:
import tensorflow as tf
import zipfile

In [ ]:
!curl -O https://storage.googleapis.com/ztm_tf_course/food_vision/pizza_steak.zip

!wget https://storage.googleapis.com/ztm_tf_course/food_vision/pizza_steak.zip
zip = zipfile.ZipFile('pizza_steak.zip')
zip.extractall()
zip.close()
zip

In [ ]:
import os
for dirpath, dirnames, filenames in os.walk('pizza_steak'):
    print(f"there are {len(dirnames)} directories and {len(filenames)} images in {dirpath}")

In [ ]:
import tensorflow as tf
print(tf.config.list_physical_devices('GPU'))


In [ ]:
import pathlib
import numpy as np
data_dir=pathlib.Path('pizza_steak/train')
class_names=np.array(sorted([item.name for item in data_dir.glob("*")]))
print(class_names)


In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import random
import os

def view_random_image(target_dir, target_class):
    target_folder = target_dir + target_class
    random_image = random.sample(os.listdir(target_folder), 1)
    print(random_image)
    img = mpimg.imread(target_folder + "/" + random_image[0])
    plt.imshow(img)
    plt.title(target_class)
    plt.axis('off')
    print(f"image shape: {img.shape}")
    return img

In [ ]:
img=view_random_image(target_dir="pizza_steak/train/", target_class="steak")

In [ ]:
tf.constant(img)

In [4]:
import tensorflow as tf
from keras_preprocessing.image import ImageDataGenerator
from keras.api.layers import Dropout, Flatten, Dense, Conv2D, MaxPooling2D
from keras.api.models import Sequential

tf.random.set_seed(42)
train_datagen = ImageDataGenerator(rescale=1. / 255)
valid_datagen = ImageDataGenerator(rescale=1. / 255)

train_dir = "/Artificial/TENSORFLOW/pizza_steak/train/"
test_dir = "/Artificial/TENSORFLOW/pizza_steak/test/"

#import data frm directories and turn it into batches
train_data = train_datagen.flow_from_directory(train_dir,
                                               batch_size=32,
                                               target_size=(224, 224),
                                               class_mode="binary",
                                               seed=42)
valid_data = valid_datagen.flow_from_directory(test_dir,
                                               batch_size=32,
                                               target_size=(224, 224),
                                               class_mode="binary",
                                               seed=42)

model1 = Sequential([
    Conv2D(filters=10, kernel_size=3, activation='relu', input_shape=(224, 224, 3)),
    Conv2D(filters=10, kernel_size=3, activation='relu'),
    MaxPooling2D(pool_size=2, padding='valid'),
    Conv2D(filters=10, kernel_size=3, activation='relu'),
    Conv2D(filters=10, kernel_size=3, activation='relu'),
    MaxPooling2D(pool_size=2),
    Flatten(),
    Dense(1, activation='sigmoid'),
])

model1.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
his1 = model1.fit(train_data, epochs=5, steps_per_epoch=len(train_data),
                  validation_data=valid_data, validation_steps=len(valid_data), verbose=2)

Found 1500 images belonging to 2 classes.
Found 500 images belonging to 2 classes.
Epoch 1/5


D:\Artificial\.venv\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


47/47 - 8s - 166ms/step - accuracy: 0.6987 - loss: 0.5589 - val_accuracy: 0.8340 - val_loss: 0.3599
Epoch 2/5
47/47 - 7s - 146ms/step - accuracy: 0.8113 - loss: 0.4124 - val_accuracy: 0.8560 - val_loss: 0.3141
Epoch 3/5
47/47 - 7s - 139ms/step - accuracy: 0.8353 - loss: 0.3823 - val_accuracy: 0.8720 - val_loss: 0.3082
Epoch 4/5
47/47 - 7s - 139ms/step - accuracy: 0.8420 - loss: 0.3752 - val_accuracy: 0.8520 - val_loss: 0.3626
Epoch 5/5
47/47 - 5s - 99ms/step - accuracy: 0.8567 - loss: 0.3471 - val_accuracy: 0.8740 - val_loss: 0.3041


In [5]:
model1.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 222, 222, 10)   │           280 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 220, 220, 10)   │           910 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 110, 110, 10)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 108, 108, 10)   │           910 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 106, 106, 10)   │           910 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 53, 53, 10)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 28090)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │        28,091 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 93,305 (364.48 KB)

 Trainable params: 31,101 (121.49 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 62,204 (242.99 KB)

In [2]:
import tensorflow as tf
from keras_preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale=1. / 255)
valid_datagen = ImageDataGenerator(rescale=1. / 255)

train_dir = "/Artificial/TENSORFLOW/pizza_steak/train/"
test_dir = "/Artificial/TENSORFLOW/pizza_steak/test/"

#import data frm directories and turn it into batches
train_data = train_datagen.flow_from_directory(train_dir,
                                               batch_size=32,
                                               target_size=(224, 224),
                                               class_mode="binary",
                                               seed=42)
valid_data = valid_datagen.flow_from_directory(test_dir,
                                               batch_size=32,
                                               target_size=(224, 224),
                                               class_mode="binary",
                                               seed=42)

Found 1500 images belonging to 2 classes.
Found 500 images belonging to 2 classes.


In [3]:
from keras.api.layers import Dropout, Flatten, Dense, Conv2D, MaxPooling2D
from keras.api.models import Sequential

tf.random.set_seed(42)

model2=Sequential([
    Flatten(input_shape=(224, 224, 3)),
    Dense(4, activation='relu'),
    Dense(4, activation='relu'),
    Dense(1, activation='sigmoid'),
])

model2.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
his2=model2.fit(train_data, epochs=5, steps_per_epoch=len(train_data),
                validation_data=valid_data, validation_steps=len(valid_data), verbose=2)

D:\Artificial\.venv\Lib\site-packages\keras\src\layers\reshaping\flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
D:\Artificial\.venv\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/5
47/47 - 9s - 197ms/step - accuracy: 0.5007 - loss: 0.7093 - val_accuracy: 0.5000 - val_loss: 0.6931
Epoch 2/5
47/47 - 4s - 87ms/step - accuracy: 0.5000 - loss: 0.6932 - val_accuracy: 0.5000 - val_loss: 0.6931
Epoch 3/5
47/47 - 4s - 75ms/step - accuracy: 0.5000 - loss: 0.6932 - val_accuracy: 0.5000 - val_loss: 0.6931
Epoch 4/5
47/47 - 4s - 76ms/step - accuracy: 0.5000 - loss: 0.6932 - val_accuracy: 0.5000 - val_loss: 0.6932
Epoch 5/5
47/47 - 4s - 77ms/step - accuracy: 0.5000 - loss: 0.6932 - val_accuracy: 0.5000 - val_loss: 0.6931


In [6]:
model2.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ flatten (Flatten)               │ (None, 150528)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 4)              │       602,116 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 4)              │            20 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │             5 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,806,425 (6.89 MB)

 Trainable params: 602,141 (2.30 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 1,204,284 (4.59 MB)

In [10]:
tf.random.set_seed(42)
model3 = Sequential([
    Flatten(input_shape=(224, 224, 3)),
    Dense(100, activation='relu'),
    Dense(100, activation='relu'),
    Dense(100, activation='relu'),
    Dense(1, activation='sigmoid'),
])

model3.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

his3 = model3.fit(train_data, epochs=5, steps_per_epoch=len(train_data), validation_data=valid_data,
                  validation_steps=len(valid_data), verbose=2)

Epoch 1/5


D:\Artificial\.venv\Lib\site-packages\keras\src\layers\reshaping\flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


47/47 - 7s - 144ms/step - accuracy: 0.6313 - loss: 1.7175 - val_accuracy: 0.6520 - val_loss: 0.9823
Epoch 2/5
47/47 - 6s - 123ms/step - accuracy: 0.7240 - loss: 0.6794 - val_accuracy: 0.7740 - val_loss: 0.4777
Epoch 3/5
47/47 - 6s - 122ms/step - accuracy: 0.6867 - loss: 0.8727 - val_accuracy: 0.7280 - val_loss: 0.6966
Epoch 4/5
47/47 - 6s - 122ms/step - accuracy: 0.7427 - loss: 0.6824 - val_accuracy: 0.7340 - val_loss: 0.7204
Epoch 5/5
47/47 - 6s - 122ms/step - accuracy: 0.7087 - loss: 0.7491 - val_accuracy: 0.7300 - val_loss: 0.6061
